Cool guide for virtual env : https://packaging.python.org/en/latest/guides/installing-using-pip-and-virtual-environments/

some important guide:
- https://spacy.io/usage
- https://huggingface.co/models
- https://pytorch.org/
- https://www.nltk.org/
- https://radimrehurek.com/gensim/index.html


some github about the project 
- https://github.com/behavioral-data/Cognitive-Reframing
- https://github.com/rbroc/NLP-AU-23/blob/main/nbs/classroom_03.ipynb
- https://medium.com/@evertongomede/clustering-text-in-natural-language-processing-unveiling-patterns-and-insights-8c3cd137b135
- https://towardsdatascience.com/clustering-sentence-embeddings-to-identify-intents-in-short-text-48d22d3bf02e



In [29]:
# imports
import spacy
import en_core_web_trf


ModuleNotFoundError: No module named 'en_core_web_trf'

In [27]:

nlp = spacy.load("en_core_web_trf")


OSError: [E050] Can't find model 'en_core_web_trf'. It doesn't seem to be a Python package or a valid path to a data directory.

In [16]:
doc = nlp("This is a sentence.")
print([(w.text, w.pos_) for w in doc])


[('This', 'PRON'), ('is', 'AUX'), ('a', 'DET'), ('sentence', 'NOUN'), ('.', 'PUNCT')]
